# Tabpfn

The following starter package takes the starting notebook from Chris (https://www.kaggle.com/code/cdeotte/xgboost-catboost-baseline-cv-668-lb-668)
and applys the newly released tabpfn with minimal feat engineering. In order to use it, you have to attach Carls tabpfn notebook (https://www.kaggle.com/code/carlmcbrideellis/tabpfn-2-0-1)

In [1]:
# Huge thanks to Chris Deotte
!pip install /kaggle/input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/autograd-gamma-0.5.0.tar.gz
!pip install /kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/formulaic-1.0.2-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/lifelines-0.30.0-py3-none-any.whl

Processing /kaggle/input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
autograd is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing /kaggle/input/pip-install-lifelines/autograd-gamma-0.5.0.tar.gz
  Preparing metadata (setup.py) ... done
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=09c56d799b1fd9fa044ee1d720110cba0b93a7ddc4984a64a92b9ede8c95800b
  Stored in directory: /root/.cache/pip/wheels/6b/b5/e0/4c79e15c0b5f2c15ecf613c720bb20daab20a666eb67135155
Successfully built autograd-gamma
Processing /kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
Processing /kaggle/input/pip-install-lifelines/formulaic-1.0.2-py3-none-any.whl
Processing /kaggle/input/pip-install-lifelines/lifelines-0.30.0-py3-none-any.whl


In [2]:
# install TABPFN thanks to Carl McBride Ellis
!pip install -q --no-index --find-links /kaggle/input/tabpfn-2-0-1 tabpfn
!mkdir -p /root/.cache/tabpfn/
!cp /kaggle/input/tabpfn-2-0-1/tabpfn-v2-regressor.ckpt /root/.cache/tabpfn/tabpfn-v2-regressor.ckpt

In [3]:
!pip install -q /kaggle/input/wheelhouse-cibmtr/scikit_learn-1.4.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

In [4]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from tabpfn import TabPFNRegressor

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

test = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/test.csv")
print("Test shape:", test.shape )

train = pd.read_excel("/kaggle/input/dataset-folding-strategy/random_folding.xlsx")
print("Train shape:",train.shape)
train.head()

Test shape: (3, 58)
Train shape: (28800, 61)


,ID,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,graft_type,vent_hist,renal_issue,pulm_severe,prim_disease_hct,hla_high_res_6,cmv_status,hla_high_res_10,hla_match_dqb1_high,tce_imm_match,hla_nmdp_6,hla_match_c_low,rituximab,hla_match_drb1_low,hla_match_dqb1_low,prod_type,cyto_score_detail,conditioning_intensity,ethnicity,year_hct,obesity,mrd_hct,in_vivo_tcd,tce_match,hla_match_a_high,hepatic_severe,donor_age,prior_tumor,hla_match_b_low,peptic_ulcer,age_at_hct,hla_match_a_low,gvhd_proph,rheum_issue,sex_match,hla_match_b_high,race_group,comorbidity_score,karnofsky_score,hepatic_mild,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10,efs,efs_time,fold
0,0,N/A - non-malignant indication,No,NaN,No,NaN,NaN,No TBI,No,6.0,Bone marrow,No,No,No,IEA,6.0,+/+,NaN,2.0,NaN,6.0,2.0,No,2.0,2.0,BM,NaN,NaN,Not Hispanic or Latino,2016,No,NaN,Yes,NaN,2.0,No,NaN,No,2.0,No,9.942,2.0,FKalone,No,M-F,2.0,More than one race,0.0,90.0,No,NaN,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0,42.356,3
1,1,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,Peripheral blood,No,No,No,AML,6.0,+/+,10.0,2.0,P/P,6.0,2.0,No,2.0,2.0,PB,Intermediate,MAC,Not Hispanic or Latino,2008,No,Positive,No,Permissive,2.0,No,72.29,No,2.0,No,43.705,2.0,Other GVHD Prophylaxis,No,F-F,2.0,Asian,3.0,90.0,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,Yes,10.0,1,4.672,7
2,2,N/A - non-malignant indication,No,NaN,No,2.0,8.0,No TBI,No,6.0,Bone marrow,No,No,No,HIS,6.0,+/+,10.0,2.0,P/P,6.0,2.0,No,2.0,2.0,BM,NaN,NaN,Not Hispanic or Latino,2019,No,NaN,Yes,NaN,2.0,No,NaN,No,2.0,No,33.997,2.0,Cyclophosphamide alone,No,F-M,2.0,More than one race,0.0,90.0,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,No,10.0,0,19.793,6
3,3,High,No,Intermediate,No,2.0,8.0,No TBI,No,6.0,Bone marrow,No,No,No,ALL,6.0,+/+,10.0,2.0,P/P,6.0,2.0,No,2.0,2.0,BM,Intermediate,MAC,Not Hispanic or Latino,2009,No,Positive,No,Permissive,2.0,No,29.23,No,2.0,No,43.245,2.0,FK+ MMF +- others,No,M-M,2.0,White,0.0,90.0,Yes,Permissive mismatched,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0,102.349,1
4,4,High,No,NaN,No,2.0,8.0,No TBI,No,6.0,Peripheral blood,No,No,No,MPN,6.0,+/+,10.0,2.0,NaN,5.0,2.0,No,2.0,2.0,PB,NaN,MAC,Hispanic or Latino,2018,No,NaN,Yes,NaN,2.0,No,56.81,No,2.0,No,29.740,2.0,TDEPLETION +- other,No,M-F,2.0,American Indian or Alaska Native,1.0,90.0,No,Permissive mismatched,Related,MEL,8.0,No,2.0,No,10.0,0,16.223,4


In [5]:
# create target
from lifelines import KaplanMeierFitter, NelsonAalenFitter, CoxPHFitter

def create_km_target(data, n_splits=5, random_state=42):
    """
    Uses Kaplan-Meier
    """
    kmf = KaplanMeierFitter()
    oof_preds = np.zeros(len(data))
    
    cv = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    for fold, (train_idx, valid_idx) in enumerate(cv.split(data)):
        train_data = data.iloc[train_idx]
        valid_data = data.iloc[valid_idx]

        kmf.fit(durations=train_data['efs_time'], event_observed=train_data['efs'])
        oof_preds[valid_idx] = kmf.survival_function_at_times(
            valid_data['efs_time']
        ).values

    data['y'] = oof_preds
    return data


def create_na_target(data, n_splits=5, random_state=42):
    """
    Uses Nelson-Aalen
    """
    naf = NelsonAalenFitter()
    oof_preds = np.zeros(len(data))

    cv = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    for fold, (train_idx, valid_idx) in enumerate(cv.split(data)):
        train_data = data.iloc[train_idx]
        valid_data = data.iloc[valid_idx]

        naf.fit(durations=train_data['efs_time'], event_observed=train_data['efs'])
        survival_vals = np.exp(-naf.cumulative_hazard_at_times(valid_data['efs_time']).values)
        oof_preds[valid_idx] = survival_vals

    data['y_na'] = oof_preds
    return data

train = create_km_target(train)
train = create_na_target(train)

In [6]:
# missing and note done as same
train = train.replace('Not done', 'missing')
test = test.replace('Not done', 'missing')

train = train.replace('Not tested', 'missing')
test = test.replace('Not tested', 'missing')

train['na_count'] = train.isna().sum(axis=1)
test['na_count'] = test.isna().sum(axis=1)

train['age_karnofsky'] = train['age_at_hct'] * train['karnofsky_score']
train['age_comorbidity'] = train['age_at_hct'] * train['comorbidity_score']
train['donor_recipient_age_diff'] = abs(train['donor_age'] - train['age_at_hct'])
train['hla_match_ratio'] = (train['hla_high_res_8'] + train['hla_low_res_8']) / 16
train['age_squared'] = train['age_at_hct'] ** 2
train['karnofsky_squared'] = train['karnofsky_score'] ** 2
train['16?'] = np.where(train['age_at_hct']<=16,1,0)

test['age_karnofsky'] = test['age_at_hct'] * test['karnofsky_score']
test['age_comorbidity'] = train['age_at_hct'] * test['comorbidity_score']
test['donor_recipient_age_diff'] = abs(test['donor_age'] - test['age_at_hct'])
test['hla_match_ratio'] = (test['hla_high_res_8'] + test['hla_low_res_8']) / 16
test['age_squared'] = test['age_at_hct'] ** 2
test['karnofsky_squared'] = test['karnofsky_score'] ** 2
test['16?'] = np.where(test['age_at_hct']<=16,1,0)


RMV = ["ID","efs","efs_time","y","y_na","fold"]
FEATURES = [c for c in train.columns if not c in RMV]
print(f"There are {len(FEATURES)} FEATURES: {FEATURES}")

There are 65 FEATURES: ['dri_score', 'psych_disturb', 'cyto_score', 'diabetes', 'hla_match_c_high', 'hla_high_res_8', 'tbi_status', 'arrhythmia', 'hla_low_res_6', 'graft_type', 'vent_hist', 'renal_issue', 'pulm_severe', 'prim_disease_hct', 'hla_high_res_6', 'cmv_status', 'hla_high_res_10', 'hla_match_dqb1_high', 'tce_imm_match', 'hla_nmdp_6', 'hla_match_c_low', 'rituximab', 'hla_match_drb1_low', 'hla_match_dqb1_low', 'prod_type', 'cyto_score_detail', 'conditioning_intensity', 'ethnicity', 'year_hct', 'obesity', 'mrd_hct', 'in_vivo_tcd', 'tce_match', 'hla_match_a_high', 'hepatic_severe', 'donor_age', 'prior_tumor', 'hla_match_b_low', 'peptic_ulcer', 'age_at_hct', 'hla_match_a_low', 'gvhd_proph', 'rheum_issue', 'sex_match', 'hla_match_b_high', 'race_group', 'comorbidity_score', 'karnofsky_score', 'hepatic_mild', 'tce_div_match', 'donor_related', 'melphalan_dose', 'hla_low_res_8', 'cardiac', 'hla_match_drb1_high', 'pulm_moderate', 'hla_low_res_10', 'na_count', 'age_karnofsky', 'age_comorb

In [7]:
CATS = []
for c in FEATURES:
    if train[c].dtype=="object":
        CATS.append(c)
        train[c] = train[c].fillna("missing")
        test[c] = test[c].fillna("missing")
print(f"In these features, there are {len(CATS)} CATEGORICAL FEATURES: {CATS}")

In these features, there are 35 CATEGORICAL FEATURES: ['dri_score', 'psych_disturb', 'cyto_score', 'diabetes', 'tbi_status', 'arrhythmia', 'graft_type', 'vent_hist', 'renal_issue', 'pulm_severe', 'prim_disease_hct', 'cmv_status', 'tce_imm_match', 'rituximab', 'prod_type', 'cyto_score_detail', 'conditioning_intensity', 'ethnicity', 'obesity', 'mrd_hct', 'in_vivo_tcd', 'tce_match', 'hepatic_severe', 'prior_tumor', 'peptic_ulcer', 'gvhd_proph', 'rheum_issue', 'sex_match', 'race_group', 'hepatic_mild', 'tce_div_match', 'donor_related', 'melphalan_dose', 'cardiac', 'pulm_moderate']


In [8]:
combined = pd.concat([train,test],axis=0,ignore_index=True)
#print("Combined data shape:", combined.shape )

# LABEL ENCODE CATEGORICAL FEATURES
print("We LABEL ENCODE the CATEGORICAL FEATURES: ",end="")
for c in FEATURES:

    # LABEL ENCODE CATEGORICAL AND CONVERT TO INT32 CATEGORY
    if c in CATS:
        print(f"{c}, ",end="")
        combined[c],_ = combined[c].factorize()
        combined[c] -= combined[c].min()
        combined[c] = combined[c].astype("int32")
        combined[c] = combined[c].astype("category")
        
    # REDUCE PRECISION OF NUMERICAL TO 32BIT TO SAVE MEMORY
    else:
        if combined[c].dtype=="float64":
            combined[c] = combined[c].astype("float32")
        if combined[c].dtype=="int64":
            combined[c] = combined[c].astype("int32")
    
train = combined.iloc[:len(train)].copy()
test = combined.iloc[len(train):].reset_index(drop=True).copy()

We LABEL ENCODE the CATEGORICAL FEATURES: dri_score, psych_disturb, cyto_score, diabetes, tbi_status, arrhythmia, graft_type, vent_hist, renal_issue, pulm_severe, prim_disease_hct, cmv_status, tce_imm_match, rituximab, prod_type, cyto_score_detail, conditioning_intensity, ethnicity, obesity, mrd_hct, in_vivo_tcd, tce_match, hepatic_severe, prior_tumor, peptic_ulcer, gvhd_proph, rheum_issue, sex_match, race_group, hepatic_mild, tce_div_match, donor_related, melphalan_dose, cardiac, pulm_moderate, 

In [9]:
%%time
FOLDS = 10
    
oof_tabpfn_kap= np.zeros(len(train))
pred_tabpfn_kap = np.zeros(len(test))
models = {}

for idx, fold in enumerate(range(FOLDS)):

    print("#"*25)
    print(f"### Fold {fold+1}")
    print("#"*25)
    
    x_train = train[train["fold"]!=fold][FEATURES].copy()
    y_train = train[train["fold"]!=fold]["y"]
    x_valid = train[train["fold"]==fold][FEATURES].copy()
    y_valid = train[train["fold"]==fold]["y"]
    x_test = test[FEATURES].copy()
    
    tabpfn_kap = TabPFNRegressor(random_state=42,ignore_pretraining_limits=True,device="cuda")
    tabpfn_kap.fit(x_train, y_train)

    # INFER OOF
    oof_tabpfn_kap[x_valid.index] = tabpfn_kap.predict(x_valid)
    # INFER TEST
    pred_tabpfn_kap += tabpfn_kap.predict(x_test)

    models[fold] = tabpfn_kap

# COMPUTE AVERAGE TEST PREDS
pred_tabpfn_kap /= FOLDS

#########################
### Fold 1
#########################


/usr/local/lib/python3.10/dist-packages/tabpfn/regressor.py:425: UserWarning: Number of samples 25920 is greater than the maximum Number of samples 10000 supported by the model. You may see degraded performance.
  X, y, feature_names_in, n_features_in = validate_Xy_fit(


#########################
### Fold 2
#########################


/usr/local/lib/python3.10/dist-packages/tabpfn/regressor.py:425: UserWarning: Number of samples 25920 is greater than the maximum Number of samples 10000 supported by the model. You may see degraded performance.
  X, y, feature_names_in, n_features_in = validate_Xy_fit(


#########################
### Fold 3
#########################


...

#########################
### Fold 9
#########################


/usr/local/lib/python3.10/dist-packages/tabpfn/regressor.py:425: UserWarning: Number of samples 25920 is greater than the maximum Number of samples 10000 supported by the model. You may see degraded performance.
  X, y, feature_names_in, n_features_in = validate_Xy_fit(


#########################
### Fold 10
#########################


/usr/local/lib/python3.10/dist-packages/tabpfn/regressor.py:425: UserWarning: Number of samples 25920 is greater than the maximum Number of samples 10000 supported by the model. You may see degraded performance.
  X, y, feature_names_in, n_features_in = validate_Xy_fit(


CPU times: user 2h 31min 51s, sys: 31 s, total: 2h 32min 22s
Wall time: 2h 36min 6s


In [10]:
from metric import score

y_true = train[["ID","efs","efs_time","race_group"]].copy()
y_pred = train[["ID"]].copy()
y_pred["prediction"] = oof_tabpfn_kap
m = score(y_true.copy(), y_pred.copy(), "ID")
print(f"\nOverall CV for Tabpfn KaplanMeier =",m)

/kaggle/usr/lib/eefs-concordance-index/metric.py:59: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged_df_race_dict = dict(merged_df.groupby(['race_group']).groups)



Overall CV for Tabpfn KaplanMeier = 0.6608750819039065


In [11]:
# %%time
# FOLDS = 3
# kf = KFold(n_splits=FOLDS, shuffle=True, random_state=42)
    
# oof_tabpfn_aal= np.zeros(len(train))
# pred_tabpfn_aal = np.zeros(len(test))

# for i, (train_index, test_index) in enumerate(kf.split(train)):

#     print("#"*25)
#     print(f"### Fold {i+1}")
#     print("#"*25)
    
#     x_train = train.loc[train_index,FEATURES].copy()
#     y_train = train.loc[train_index,"y_na"]
#     x_valid = train.loc[test_index,FEATURES].copy()
#     y_valid = train.loc[test_index,"y_na"]
#     x_test = test[FEATURES].copy()
#     tabpfn_aal = TabPFNRegressor(random_state=42,ignore_pretraining_limits=True,device="cuda")
#     tabpfn_aal.fit(x_train, y_train)

#     # INFER OOF
#     oof_tabpfn_aal[test_index] = tabpfn_aal.predict(x_valid)
#     # INFER TEST
#     pred_tabpfn_aal += tabpfn_aal.predict(x_test)

# # COMPUTE AVERAGE TEST PREDS
# pred_tabpfn_aal /= FOLDS

In [12]:
# from metric import score

# y_true = train[["ID","efs","efs_time","race_group"]].copy()
# y_pred = train[["ID"]].copy()
# y_pred["prediction"] = oof_tabpfn_aal
# m = score(y_true.copy(), y_pred.copy(), "ID")
# print(f"\nOverall CV for Tabpfn AAlon =",m)

In [13]:
from scipy.stats import rankdata 

y_true = train[["ID","efs","efs_time","race_group"]].copy()
y_pred = train[["ID"]].copy()
y_pred["prediction"] = rankdata(oof_tabpfn_kap) #+ rankdata(oof_tabpfn_aal)
m = score(y_true.copy(), y_pred.copy(), "ID")
print(f"\nOverall CV for TabFn target Ensemble =",m)

/kaggle/usr/lib/eefs-concordance-index/metric.py:59: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged_df_race_dict = dict(merged_df.groupby(['race_group']).groups)



Overall CV for TabFn target Ensemble = 0.6608750819039065


In [14]:
sub = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/sample_submission.csv")
sub.prediction = rankdata(pred_tabpfn_kap) #+ rankdata(pred_tabpfn_aal)

sub.to_csv("submission.csv",index=False)
print("Sub shape:",sub.shape)
sub.head()

Sub shape: (3, 2)


,ID,prediction
0,28800,2.0
1,28801,3.0
2,28802,1.0


In [15]:
import pickle
with open('tf_exp_01.pkl', 'wb') as f:
    pickle.dump(models, f)

In [16]:
train['predictions'] = oof_tabpfn_kap
train.to_excel("tf_exp_01_oof.xlsx")